In [ ]:
F=QQ; T.<x,y,z>=PolynomialRing(F,order='degrevlex')

# The following function inputs a homogeneous quartic in QQ[x,y,z],
# and outputs the 28 bitangents
def compute_bitangents(f):
    F=QQ; T.<x,y,z>=PolynomialRing(F)
    
    # Check whether the quartic is smooth
    Grad=ideal(f,diff(f,x),diff(f,y),diff(f,z))
    if not Grad.dimension()==0:
        sys.exit("Quartic is not smooth!")

    # Create a new polynomial ring with variables a,b (for the bitangent) and a0,..,a4 (TODO)
    R.<x,y,z,a,b,a0,a1,a2,a3,a4>=PolynomialRing(F)
    f0=f.base_extend(R)
    S.<a,b>=PolynomialRing(F)
    digits=50
    threshold=0.000000000001
    almostzero=threshold

    # Take an abstract line with equation ax+by+z
    Line= a*x+b*y+z;
    puresquare=ideal(a0*a3^2-a1^2*a4,8*a0^2*a3-4*a0*a1*a2+a1^3,8*a1*a4^2-4*a2*a3*a4+a3^3,8*a0*a1*a4-4*a0*a2*a3+a1^2*a3,8*a0*a3*a4-4*a1*a2*a4+a1*a3^2,16*a0^2*a4+2*a0*a1*a3-4*a0*a2^2+a1^2*a2,16*a0*a4^2+2*a1*a3*a4-4*a2^2*a4+a2*a3^2);

    # Consider the resultant of the equation of the line with respect to z.
    Res=f0.resultant(Line,z)    
    Res=Res.subs(y=1)    
    phi=hom(R,S,[0,0,0,a,b,Res.coefficient({x:0}),Res.coefficient({x:1}),Res.coefficient({x:2}),Res.coefficient({x:3}),Res.coefficient({x:4})])
    
    bit1 = phi(puresquare)     
    
    I_ideal=singular.groebner(singular(bit1))
    singular.lib('solve.lib')
    VRing=singular.solve(I_ideal,digits)
    singular.set_ring(VRing)
    B1=singular("SOL")
    
    nreal1=0
    Bitangents=[]
    RealBitangents=[]
    for k in [1..len(B1)]:
        real=0;
        if ((B1[k][1].impart()).absValue()<threshold) and ((B1[k][2].impart()).absValue()<threshold): 
            real=1
            RealBitangents=RealBitangents+[(float(B1[k][1].repart())+float(B1[k][1].impart())*i)*x+(float(B1[k][2].repart())+float(B1[k][2].impart())*i)*y+z]
        nreal1=nreal1+real
        Bitangents=Bitangents+[(float(B1[k][1].repart())+float(B1[k][1].impart())*i)*x+(float(B1[k][2].repart())+float(B1[k][2].impart())*i)*y+z]
    Line=a*x+y     
    Res=f0.resultant(Line,y)    
    Res=Res.subs(z=1)   
    phi=hom(R,S,[0,0,0,a,0,Res.coefficient({x:0}),Res.coefficient({x:1}),Res.coefficient({x:2}),Res.coefficient({x:3}),Res.coefficient({x:4})])  
    bit2=phi(puresquare)+ideal(b)
    
    if dimension(bit2)==-1: nreal2=0
    else: 
          I_ideal=singular.groebner(singular(bit2))
          singular.lib('solve.lib')
          VRing=singular.solve(I_ideal,digits)
          singular.set_ring(VRing)
          B2=singular("SOL")
          nreal2=0
          for k in [1..len(B2)]:
                real=0
                if ((B2[k][1].impart()).absValue()<threshold) and ((B2[k][2].impart()).absValue()<threshold):
                    real=1
                    RealBitangents=RealBitangents+[(float(B2[k][1].repart())+float(B2[k][1].impart())*i)*x+y]
                nreal2=nreal2+real  
                Bitangents=Bitangents+[(float(B2[k][1].repart())+float(B2[k][1].impart())*i)*x+y]
    
    Res=f0.resultant(x)
    Res=Res.subs(z=1)
    phi=hom(R,F,[0,0,0,0,0,Res.coefficient({y:0}),Res.coefficient({y:1}),Res.coefficient({y:2}),Res.coefficient({y:3}),Res.coefficient({y:4})])  
    bit3=phi(puresquare)
    if bit3==ideal(0):
        nreal3=1
        Bitangents=Bitangents+[x]
        RealBitangents=RealBitangents+[x]
    else: nreal3=0
    
    NRealBit=nreal1+nreal2+nreal3
    if len(Bitangents)!=28:
        return OSError("Something has gone wrong. We found "+str(len(Bitangents))+" bitangents")
    # print("The quartic has 28 bitangets, stored in 'Bitangents', and "+str(NRealBit)+" real bitangents, stored in 'RealBitangents'.")
    
    return Bitangents